In [9]:
# Import packages.
import pandas as pd
import numpy as np
from datetime import timedelta

In [13]:
# Read in the data.
crypto = pd.read_pickle("../data/3_9_crypto.pkl")
tweets = pd.read_pickle("../data/3_24_bert_data.pkl")

In [14]:
# Define a function.
def get_max(min_stamp, price):
    max_stamp = min_stamp + timedelta(hours=12)  ## CHANGE HERE.
    mask = (crypto["stamp"] > min_stamp) & (crypto["stamp"] < max_stamp)
    sub_df = crypto.loc[mask]
    max_price = sub_df.price.max()

    # Create the difference.
    difference = (max_price - price) / price
    
    # Return. CHANGE HERE.
    if difference > .0025:
        return 1
    else:
        return 0

# Apply the function.
crypto["increase"] = crypto.apply(lambda x: get_max(x[0], x[2]), axis=1)

In [15]:
# Join the data.
training_data = tweets.merge(crypto, left_on="time", right_on="stamp")
training_data = training_data.set_index("tweet_id")
training_data = training_data.loc[:, ["time", "tweet", "embedding", "increase"]]

# Write the file.
training_data.to_pickle("../data/3_25_training_data.pkl")